In [1]:
from pyspark.sql import SparkSession
import csv
import pandas as pd

spark = SparkSession.builder.master("local").appName("lopputyo").getOrCreate()

df = spark.read.csv("merged0_21.csv", header=True, inferSchema=True)
df.show(vertical=True)

df.registerTempTable("example")


-RECORD 0-----------------------------
 id                 | 0               
 aikatyyppi         | Vuosi           
 kuukausi_nro       | 12              
 vuosikuukausi      | 200312          
 vuosi              | 2003            
 kunta_nro          | 257             
 kunta_nimi         | Kirkkonummi     
 ikaryhma           | 35-39           
 sukupuoli          | Nainen          
 etuus              | Työmarkkinatuki 
 korvausperuste     | Yhteensä        
 saaja_lkm          | 45              
 korvattu_paiva_lkm | 8277            
 maksettu_eur       | 215964.47       
-RECORD 1-----------------------------
 id                 | 1               
 aikatyyppi         | Vuosi           
 kuukausi_nro       | 12              
 vuosikuukausi      | 200312          
 vuosi              | 2003            
 kunta_nro          | 256             
 kunta_nimi         | Kinnula         
 ikaryhma           | 35-39           
 sukupuoli          | Nainen          
 etuus              | Työ

c:\Lipasto\MassData23\mass_data_analysis_2023\venv\lib\site-packages\pyspark\sql\dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [2]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- aikatyyppi: string (nullable = true)
 |-- kuukausi_nro: integer (nullable = true)
 |-- vuosikuukausi: integer (nullable = true)
 |-- vuosi: integer (nullable = true)
 |-- kunta_nro: integer (nullable = true)
 |-- kunta_nimi: string (nullable = true)
 |-- ikaryhma: string (nullable = true)
 |-- sukupuoli: string (nullable = true)
 |-- etuus: string (nullable = true)
 |-- korvausperuste: string (nullable = true)
 |-- saaja_lkm: integer (nullable = true)
 |-- korvattu_paiva_lkm: integer (nullable = true)
 |-- maksettu_eur: double (nullable = true)



In [3]:
df.groupBy('etuus').count().orderBy('count',ascending=False).show()
df.groupBy('korvausperuste').count().orderBy('count',ascending=False).show(50)

+--------------------+-------+
|               etuus|  count|
+--------------------+-------+
|            Yhteensä|2915766|
|Työmarkkinatukiet...|2756622|
|     Työmarkkinatuki|2719127|
|      Peruspäiväraha|1010349|
|      Kotoutumistuki|  75271|
|      Yhdistelmätuki|  55317|
+--------------------+-------+

+--------------------+-------+
|      korvausperuste|  count|
+--------------------+-------+
|            Yhteensä|2475997|
|    Työttömyysajalta|2267018|
|Työllistymistä ed...|1513757|
|-Kuntouttava työt...| 875424|
|   -Työvoimakoulutus| 626443|
|-Omaehtoinen opis...| 553266|
|         -Työkokeilu| 436926|
|       Tieto puuttuu| 149914|
|  -Työnhakuvalmennus| 143439|
|       -Uravalmennus| 133852|
| -Tuntematon palvelu| 102603|
|  -Työelämävalmennus|  75923|
|     -Yhdistelmätuki|  55317|
|     -Työharjoittelu|  52594|
|-Avo- tai kuntout...|  21241|
|-Rinnasteinen kou...|  16287|
|      -Matka-avustus|  13611|
|    -Koulutuskokeilu|  11584|
|    -Työelämäkokeilu|   6377|
|-Työ- 

In [2]:
new_df = df.withColumn("maksu_per_hlo", (df["maksettu_eur"]/df["saaja_lkm"])/12).orderBy('id',ascending=False).filter((df["aikatyyppi"] == "Vuosi"))
new_df = new_df.filter(new_df["maksettu_eur"] > 0)
new_df.show()


+-------+----------+------------+-------------+-----+---------+-----------+--------+---------+--------------+----------------+---------+------------------+------------+------------------+
|     id|aikatyyppi|kuukausi_nro|vuosikuukausi|vuosi|kunta_nro| kunta_nimi|ikaryhma|sukupuoli|         etuus|  korvausperuste|saaja_lkm|korvattu_paiva_lkm|maksettu_eur|     maksu_per_hlo|
+-------+----------+------------+-------------+-----+---------+-----------+--------+---------+--------------+----------------+---------+------------------+------------+------------------+
|9355733|     Vuosi|          12|       202212| 2022|      678|      Raahe|   45-49|   Nainen|Peruspäiväraha|Työttömyysajalta|       13|              1007|    32151.64| 206.1002564102564|
|9355732|     Vuosi|          12|       202212| 2022|      680|     Raisio|   45-49|   Nainen|Peruspäiväraha|Työttömyysajalta|       13|              1521|    44221.29| 283.4698076923077|
|9355731|     Vuosi|          12|       202212| 2022|      6

In [5]:
new_df.select("saaja_lkm", "maksettu_eur", "maksu_per_hlo").describe().show()

+-------+------------------+-----------------+--------------------+
|summary|         saaja_lkm|     maksettu_eur|       maksu_per_hlo|
+-------+------------------+-----------------+--------------------+
|  count|           1978652|          1978652|             1978652|
|   mean|27.794614212099955|111642.4409874503|    287.226645767111|
| stddev| 95.24945749808242|471120.7129070449|  149.27381797607882|
|    min|                 1|             0.05|8.333333333333334E-4|
|    max|              5931|    2.597016758E7|  1415.9533333333331|
+-------+------------------+-----------------+--------------------+



In [6]:
new_df.select("kuukausi_nro", "maksu_per_hlo").groupBy('kuukausi_nro').avg().orderBy("kuukausi_nro").show()

+------------+-----------------+------------------+
|kuukausi_nro|avg(kuukausi_nro)|avg(maksu_per_hlo)|
+------------+-----------------+------------------+
|          12|             12.0|  287.226645767111|
+------------+-----------------+------------------+



In [7]:
new_df.select("ikaryhma", "maksu_per_hlo").groupBy('ikaryhma').avg().orderBy("ikaryhma").show()

+-------------+------------------+
|     ikaryhma|avg(maksu_per_hlo)|
+-------------+------------------+
|        17-19| 150.4140945758029|
|        20-24|201.21046489471163|
|        25-29|282.23715280767897|
|        30-34| 321.1767869393682|
|        35-39| 334.5714554567832|
|        40-44| 336.5791988148505|
|        45-49| 331.3207671147298|
|        50-54| 324.7212561934267|
|        55-59|343.04215603846256|
|        60-64| 395.8199180929371|
|        65-67|271.48661670605026|
|Tieto puuttuu|234.44209702719496|
+-------------+------------------+



In [3]:
new_df.select("vuosi", "maksu_per_hlo").groupBy('vuosi').avg().orderBy("vuosi").show(22)

data_year_2000 = new_df.filter(new_df["vuosi"] == "2000")
data_year_2010 = new_df.filter(new_df["vuosi"] == "2010")
data_year_2021 = new_df.filter(new_df["vuosi"] == "2021")

+-----+----------+------------------+
|vuosi|avg(vuosi)|avg(maksu_per_hlo)|
+-----+----------+------------------+
| 2000|    2000.0|192.00809564816706|
| 2001|    2001.0| 201.9509538024688|
| 2002|    2002.0|222.55850548507235|
| 2003|    2003.0| 234.3988717893467|
| 2004|    2004.0|241.78849688453406|
| 2005|    2005.0|234.49720852214753|
| 2006|    2006.0|   223.52475619383|
| 2007|    2007.0|223.73164824250847|
| 2008|    2008.0|234.27858544566527|
| 2009|    2009.0| 258.1918235178004|
| 2010|    2010.0|258.78042817400905|
| 2011|    2011.0|266.99741808989273|
| 2012|    2012.0| 321.1008141191748|
| 2013|    2013.0| 334.8632759627316|
| 2014|    2014.0| 348.9898400182696|
| 2015|    2015.0|348.79164028632994|
| 2016|    2016.0|361.12227259880785|
| 2017|    2017.0| 350.5099880222053|
| 2018|    2018.0| 342.0641652083216|
| 2019|    2019.0| 347.7609834847609|
| 2020|    2020.0|365.77452194952997|
| 2021|    2021.0| 388.8646910709369|
+-----+----------+------------------+
only showing

In [11]:
new_df.select("Kunta_nimi", "maksu_per_hlo").groupBy("Kunta_nimi").avg().orderBy("avg(maksu_per_hlo)", ascending=False).show(300)



+------------------+------------------+
|        Kunta_nimi|avg(maksu_per_hlo)|
+------------------+------------------+
|             Vöyri|393.95929361486964|
|         Raasepori| 372.0282082140798|
|   Mänttä-Vilppula| 371.4325547292736|
|          Myrskylä| 355.6133158378318|
|              Akaa|352.00526925408775|
|             Hanko|350.30667584088417|
|            Karvia| 350.1015350186216|
|        Siikalatva| 349.7970640650023|
|         Siikainen|348.95465530496864|
|         Vesilahti|348.84146723204736|
|        Lapinjärvi|  345.037463870714|
|            Multia|343.64199450273185|
|         Siikajoki|343.52272392784107|
|            Urjala|342.47722947471124|
|             Ypäjä|342.29179729433923|
|          Marttila|342.09867563274594|
|             Inkoo| 338.1112546849901|
|          Koski Tl| 334.2627306596237|
|           Ruovesi|333.69799721702464|
|        Kauniainen| 333.6264557548907|
|           Siuntio|331.77468750242343|
|           Nakkila| 331.7080792634798|


In [12]:
data_year_2000.select("Kunta_nimi", "maksu_per_hlo").groupBy("Kunta_nimi").avg().orderBy("avg(maksu_per_hlo)", ascending=False).show(300)

+------------------+------------------+
|        Kunta_nimi|avg(maksu_per_hlo)|
+------------------+------------------+
|               279|  264.885845117845|
|               943|  260.920356565698|
|               424| 255.7380746688827|
|               101|254.89796296296285|
|               776|252.92733085004696|
|       Hirvensalmi|248.61170549755457|
|               259| 242.3198299870875|
|           Kustavi|241.75775458538672|
|             Lavia|240.12456106161824|
|               728| 239.5836803810554|
|               266|238.27396141236932|
|           Kärkölä|236.95268121973422|
|               979|236.41822457235747|
|          Myrskylä|236.16631093777627|
|               633|235.31655768409001|
|               252|235.06595553381234|
|             Luvia| 234.2809992974676|
|         Konnevesi|233.98199640997527|
|               770| 233.8092628572718|
|         Pyhäranta|233.66000520110947|
|          Humppila| 231.8379484782321|
|               856| 231.6353039675357|


In [14]:
data_year_2010.select("Kunta_nimi", "maksu_per_hlo").groupBy("Kunta_nimi").avg().orderBy("avg(maksu_per_hlo)", ascending=False).show(300)

+------------------+------------------+
|        Kunta_nimi|avg(maksu_per_hlo)|
+------------------+------------------+
|            Multia|357.83192248949985|
|         Jämijärvi|346.79817624083023|
|         Virolahti| 335.3324587630451|
|           Utsjoki| 329.2394786906092|
|         Siikajoki| 325.2819979371301|
|             Lavia| 318.3381633550416|
|               559| 316.3927577053363|
|            Kihniö| 313.3903248744759|
|           Kustavi| 312.6243401650432|
|         Alavieska| 310.0129677891911|
|         Kuhmoinen|  309.998693905099|
|             Ypäjä| 308.3511448124836|
|          Maalahti|  307.842360072496|
|          Eurajoki|307.01869700381184|
|        Miehikkälä|305.80267030666704|
|          Toivakka| 305.0178583712908|
|           Kärkölä| 304.1744530907001|
|             Sysmä| 303.6126111483947|
|           Ruovesi|302.83261234312823|
|         Juupajoki| 302.2181656365479|
|           Parkano|301.88567296135375|
|         Raasepori|  300.327745868956|


In [15]:
data_year_2021.select("Kunta_nimi", "maksu_per_hlo").groupBy("Kunta_nimi").avg().orderBy("avg(maksu_per_hlo)", ascending=False).show(300)

+------------------+------------------+
|        Kunta_nimi|avg(maksu_per_hlo)|
+------------------+------------------+
|       Punkalaidun| 536.3643600350795|
|             Hanko| 496.7966886835014|
|          Myrskylä|494.06418486223777|
|            Kihniö| 492.3776180911729|
|             Sysmä|489.05221655307736|
|           Isojoki| 487.0441365116118|
|           Parkano|485.85804354223535|
|             Kaavi| 484.6126004891961|
|        Ristijärvi|484.55923282967035|
|         Siikainen|484.49222689652424|
|             Inkoo|480.20502879347254|
|         Jokioinen| 477.7812114292841|
|           Nakkila| 474.2319232874509|
|            Urjala| 472.1568456680957|
|             Ypäjä| 471.8048143102378|
|          Marttila|468.28651793135583|
|        Hyrynsalmi| 467.9084687881086|
|          Kokemäki|467.34601212666377|
|          Humppila|  467.101273041086|
|          Koski Tl|466.77594848722293|
|        Lapinjärvi| 464.7047043899269|
|         Juupajoki|462.34809140924887|


In [4]:
data_for_json = new_df.select("Kunta_nro", "maksu_per_hlo").groupBy("Kunta_nro").avg().orderBy("avg(Kunta_nro)", ascending=True)
data_for_json.show()

data_for_json_2000 = data_year_2000.select("Kunta_nro", "maksu_per_hlo").groupBy("Kunta_nro").avg().orderBy("avg(Kunta_nro)", ascending=True)
data_for_json_2010 = data_year_2010.select("Kunta_nro", "maksu_per_hlo").groupBy("Kunta_nro").avg().orderBy("avg(Kunta_nro)", ascending=True)
data_for_json_2021 = data_year_2021.select("Kunta_nro", "maksu_per_hlo").groupBy("Kunta_nro").avg().orderBy("avg(Kunta_nro)", ascending=True)



+---------+--------------+------------------+
|Kunta_nro|avg(Kunta_nro)|avg(maksu_per_hlo)|
+---------+--------------+------------------+
|        4|           4.0| 183.8373662637761|
|        5|           5.0|276.29517142495416|
|        6|           6.0|236.77741819236573|
|        9|           9.0| 315.2274844334803|
|       10|          10.0|  294.016897300876|
|       14|          14.0|221.00403761671336|
|       15|          15.0|235.32725747958423|
|       16|          16.0|293.91328420784885|
|       17|          17.0| 213.4426257181157|
|       18|          18.0|317.32614347916217|
|       19|          19.0| 319.4828405412542|
|       20|          20.0|352.00526925408775|
|       35|          35.0|295.63473214285716|
|       40|          40.0|226.72571793635515|
|       43|          43.0| 199.0369097222222|
|       44|          44.0|232.40324546198332|
|       45|          45.0|238.27712760597854|
|       46|          46.0|297.11831137115297|
|       47|          47.0|  262.03

In [5]:
data_for_json_2000.show()
data_for_json_2010.show()
data_for_json_2021.show()

+---------+--------------+------------------+
|Kunta_nro|avg(Kunta_nro)|avg(maksu_per_hlo)|
+---------+--------------+------------------+
|        4|           4.0|171.16537652098637|
|        5|           5.0|182.98090677297776|
|        6|           6.0|223.68693014182264|
|        9|           9.0| 211.0830803496021|
|       10|          10.0|  176.059278849622|
|       14|          14.0|221.00403761671336|
|       15|          15.0|193.16837899346345|
|       16|          16.0| 204.1947248386008|
|       17|          17.0| 182.6094074906948|
|       18|          18.0|198.20551722492291|
|       19|          19.0|208.19008771229215|
|       40|          40.0| 184.5636674817013|
|       44|          44.0| 192.6294707656763|
|       45|          45.0|184.96096835224685|
|       46|          46.0|217.82729229482374|
|       47|          47.0| 193.3472484322682|
|       49|          49.0|184.30314621740564|
|       50|          50.0|186.52183106926938|
|       51|          51.0|216.7938

In [7]:
import json
with open("kunnat.json", "r") as kunnat:
    kuntajson = json.load(kunnat)
    

In [17]:



string = "{\n"
for row in data_for_json.rdd.collect():
    val1 = row["Kunta_nro"]
    val2 = row["avg(maksu_per_hlo)"]
    if f'area{val1:03d}' in kuntajson:
        string += f'"area{val1:03d}": "{val2:.2f}",\n'
string += "\n}"
#print(string)


with open("kunnat2.json", "w") as kunnat2:
    json.dump(string, kunnat2)

In [144]:
jsondict = {}

for row in data_for_json.rdd.collect():
    val1 = row["Kunta_nro"]
    val2 = row["avg(maksu_per_hlo)"]
    #if f'area{val1:03d}' in kuntajson:
        #string += f'"area{val1:03d}": "{val2:.2f}",\n'
    jsondict[f'area{val1:03d}'] = f'{val2:.2f}'

with open("kunnat_average_alltime.json", "w") as kunnat3:
    json.dump(jsondict, kunnat3)

In [14]:
jsondict = {}

for row in data_for_json_2000.rdd.collect():
    val1 = row["Kunta_nro"]
    val2 = row["avg(maksu_per_hlo)"]
    #if f'area{val1:03d}' in kuntajson:
        #string += f'"area{val1:03d}": "{val2:.2f}",\n'
    jsondict[f'area{val1:03d}'] = f'{val2:.2f}'

with open("kunnat_average_2000.json", "w") as kunnat3:
    json.dump(jsondict, kunnat3)

In [15]:
jsondict = {}

for row in data_for_json_2010.rdd.collect():
    val1 = row["Kunta_nro"]
    val2 = row["avg(maksu_per_hlo)"]
    #if f'area{val1:03d}' in kuntajson:
        #string += f'"area{val1:03d}": "{val2:.2f}",\n'
    jsondict[f'area{val1:03d}'] = f'{val2:.2f}'

with open("kunnat_average_2010.json", "w") as kunnat3:
    json.dump(jsondict, kunnat3)

In [16]:
jsondict = {}

for row in data_for_json_2021.rdd.collect():
    val1 = row["Kunta_nro"]
    val2 = row["avg(maksu_per_hlo)"]
    #if f'area{val1:03d}' in kuntajson:
        #string += f'"area{val1:03d}": "{val2:.2f}",\n'
    jsondict[f'area{val1:03d}'] = f'{val2:.2f}'

with open("kunnat_average_2021.json", "w") as kunnat3:
    json.dump(jsondict, kunnat3)

In [24]:
jsondict = {}

for row1 in data_for_json_2000.rdd.collect():
    kunta1 = row1["Kunta_nro"]
    for row2 in data_for_json_2010.rdd.collect():
        kunta2 = row2["Kunta_nro"]

        #print(f"Comparing: {kunta1} and {kunta2}")
        avg1 = row1["avg(maksu_per_hlo)"]
        avg2 = row2["avg(maksu_per_hlo)"]
        
        if (kunta1 == kunta2):
            #print("Its a match!")
            sum = avg2 - avg1
            jsondict[f'area{kunta1:03d}'] = f'{sum:.2f}'
        
        
with open("kunnat_average_comparison_2000-2010.json", "w") as kunnat3:
    json.dump(jsondict, kunnat3)
        #val1 = row["Kunta_nro"]
        #val2 = row["avg(maksu_per_hlo)"]

In [25]:
jsondict = {}

for row1 in data_for_json_2010.rdd.collect():
    kunta1 = row1["Kunta_nro"]
    for row2 in data_for_json_2021.rdd.collect():
        kunta2 = row2["Kunta_nro"]

        #print(f"Comparing: {kunta1} and {kunta2}")
        avg1 = row1["avg(maksu_per_hlo)"]
        avg2 = row2["avg(maksu_per_hlo)"]
        
        if (kunta1 == kunta2):
            #print("Its a match!")
            sum = avg2 - avg1
            jsondict[f'area{kunta1:03d}'] = f'{sum:.2f}'
        
        
with open("kunnat_average_comparison_2010-2021.json", "w") as kunnat3:
    json.dump(jsondict, kunnat3)
        #val1 = row["Kunta_nro"]
        #val2 = row["avg(maksu_per_hlo)"]